In [228]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, timedelta
from time import time

from sklearn.cluster import KMeans
from matplotlib import pyplot as plt

Very_Start = time()

### DEFINITION DE TOUS LES PARAMETRES

In [248]:
# @param Does it save the figure ?
save_fig = True


# @param Connexion à la DB
driver   = '{ODBC Driver 17 for SQL Server}'
server   = 'E5591-9S6YMV2'
username = 'WVN\k.dubrulle'
login    = ('DRIVER=' + driver + 
         ';SERVER='   + server + 
         ';Trusted_Connection=yes;')


# @param Definition des table et colonnes --> Dépendent de la DB
DB             = "Archive2018_2019_prod"
TABLE_CONSO    = "Consommateur"
TABLE_ENTETE   = "Recette_entete_ticket"
TEMPS_ENTETE   = TABLE_ENTETE               + ".TEMPS.KEY_Date_reel_de_la_transaction"
NUMERO_ENTETE  = TABLE_ENTETE               + ".Numero_ticket"
TABLE_DETAIL   = "Recette_detail_ticket"
TEMPS_DETAIL   = TABLE_DETAIL               + ".TEMPS.KEY_Date_reel_transaction"
NUMERO_DETAIL  = TABLE_DETAIL               + ".Numero_ticket"
TABLE_ARTICLE  = "Article_meti_HIST"
TABLE_UNIVERS  = "Univers"


# @param Variable influançant la sélection des données
nbr_client     = -1           # Nombre de clients à sélectionner. MAX = 148188. -1 --> ALL
client_tresh   = 20           # Nombre d'article que le client doit avoir acheter pour être mis dans un segment
univers_tresh  = 10           # Nombre d'article acheté de l'univers pour qu'il soit pris en compte

Standard_Norm  = False        # Défini si on utilise la Normalization Standard
Standard_Tresh = 1            # Seuil d'acceptabilité des cluster en normalization standard
MinMax_Norm    = True         # Défini si on utilise la Normalization Minmax
MinMax_Tesh    = .3           # Seuil d'acceptabilité des cluster par défaut

do_Elbow       = False        # Est-ce qu'on fait l'analyse des Elbow ou pas ?
Kmin           = 2            # Nombre de cluster au début de l'analyse
Kmax           = 50           # Nombre de cluster à la fin de l'analyse

K              = 21           # Nombre de segment. K = 21 -> 10% client d'exclu ; K = 15 -> 15% client exclu ; K = 9 -> 20% exclu 
n_init         = 10           # Nombre de run du KMean pour trouver le meilleur min

In [230]:
# TEST Connexion
conn = pyodbc.connect(login)

df = pd.read_sql("SELECT TOP(10) ["+TABLE_CONSO+".KEY] FROM ["+DB+"].[DWH].[Dim_"+TABLE_CONSO+"]", conn)
print(df)

conn.close()

   CONSOMMATEUR.KEY  Consommateur.Numero_consommateur_freedelity  \
0                -1                                           -1   
1                 1                                          100   
2                 2                                      1000138   
3                 3                                      1000303   
4                 4                                      1000323   
5                 5                                      1000350   
6                 6                                      1000371   
7                 7                                      1000484   
8                 8                                      1000571   
9                 9                                       100058   

   Consommateur.Points  Consommateur.Visits  Consommateur.TotalCA  \
0                    0                    0                  0.00   
1                  674                    0               1089.39   
2                  287                    0 

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


# SEGMENTATION DE LA CLIENTELE

## Récupération des données correspondant aux clients

In [232]:
start = time()

sql_command = '''SELECT [''' + TABLE_ARTICLE + '''.''' + TABLE_UNIVERS + '''.KEY] AS ''' + TABLE_UNIVERS + '''
,[''' + TABLE_ENTETE + '''.''' + TABLE_CONSO + '''.KEY] AS ''' + TABLE_CONSO + '''
FROM ['''       + DB + '''].[DWH].[Fact_''' + TABLE_DETAIL + ''']
INNER JOIN [''' + DB + '''].[DWH].[Fact_''' + TABLE_ENTETE + '''] 
ON    [''' + TEMPS_DETAIL  +'''] = [''' + TEMPS_ENTETE  + '''] 
AND   [''' + NUMERO_DETAIL +'''] = [''' + NUMERO_ENTETE + ''']
INNER JOIN [''' + DB + '''].[DWH].[Dim_''' + TABLE_ARTICLE + ''']
ON ['''    + TABLE_DETAIL  + '''.'''    + TABLE_ARTICLE + '''.KEY] = [''' + TABLE_ARTICLE + '''.KEY] 
WHERE [''' + TABLE_ENTETE  + '''.'''    + TABLE_CONSO   + '''.KEY] > 0 '''
if nbr_client != -1:
    sql_command +=  ''' AND [''' + TABLE_ENTETE  +'''.'''     + TABLE_CONSO   + '''.KEY] < '''  + str(nbr_client)

print(sql_command)

conn = pyodbc.connect(login)
df_univers = pd.read_sql(sql_command, conn)
conn.close()

# environ 3min
print("Time Taken to load DB: ~" + str(int(time() - start)) + "secondes") 
nbr_data = len(df_univers)
print(nbr_data)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


SELECT [Article_meti_HIST.Univers.KEY] AS Univers
,[Recette_entete_ticket.Consommateur.KEY] AS Consommateur
FROM [Archive2018_2019_prod].[DWH].[Fact_Recette_detail_ticket]
INNER JOIN [Archive2018_2019_prod].[DWH].[Fact_Recette_entete_ticket] 
ON    [Recette_detail_ticket.TEMPS.KEY_Date_reel_transaction] = [Recette_entete_ticket.TEMPS.KEY_Date_reel_de_la_transaction] 
AND   [Recette_detail_ticket.Numero_ticket] = [Recette_entete_ticket.Numero_ticket]
INNER JOIN [Archive2018_2019_prod].[DWH].[Dim_Article_meti_HIST]
ON [Recette_detail_ticket.Article_meti_HIST.KEY] = [Article_meti_HIST.KEY] 
WHERE [Recette_entete_ticket.Consommateur.KEY] > 0 
Time Taken to load DB: ~172secondes
10836528


In [242]:
start = time()
dict_client_univ = {}

for i in range(nbr_data):
    univ   = df_univers[TABLE_UNIVERS][i]
    client = df_univers[TABLE_CONSO][i]
    if univ == -1:
        continue
    
    if not client in dict_client_univ:
        dict_client_univ[client] = {}
        
    if not univ in dict_client_univ[client]:
        dict_client_univ[client][univ] = 0
    dict_client_univ[client][univ] += 1

to_remove = []
for k,v in dict_client_univ.items():
    if sum(v.values()) < client_tresh:
        to_remove.append(k)
    else:
        maxi = max(v.values())
        dict_client_univ[k] = {key:value/maxi for key,value in v.items()}

for k in to_remove:
    del dict_client_univ[k]
        
    
clients_data = pd.DataFrame(index = dict_client_univ.keys(),
                        data = dict_client_univ.values())
clients_data = clients_data.replace(float('NaN'), 0)
 
print(clients_data)

# environ 10s * nbr_client/10.000
print("Time Taken to load Dataframe: ~" + str(int(time() - start)) + "secondes")

             344       144       148       151       461       328       183  \
11377   0.010753  0.026882  0.016129  0.048387  0.284946  0.134409  0.048387   
30766   0.048193  0.072289  0.048193  0.000000  0.144578  0.012048  0.084337   
62982   0.076923  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
97964   0.139130  0.026087  0.017391  0.017391  0.104348  0.034783  0.095652   
61405   0.020000  0.240000  0.020000  0.060000  0.000000  0.180000  0.020000   
...          ...       ...       ...       ...       ...       ...       ...   
63427   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  1.000000   
147844  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
147454  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
144632  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
145875  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.250000   

             403       418       331  .

In [249]:
# Standard Normalization
def standard_normalization(df):
    return df.iloc[:,:].apply(lambda x: (x-x.mean())/ x.std(), axis=0)

# Min-Max Normalization
from sklearn.preprocessing import MinMaxScaler
def minmax_normalization(df):
    scaler = MinMaxScaler() 
    scaled_values = scaler.fit_transform(df) 
    return pd.DataFrame(index = df.index, columns = df.columns,data = scaled_values)

norm_clients_data = clients_data
if Standard_Norm:
    norm_clients_data = standard_normalization(norm_clients_data)
if MinMax_Norm:
    norm_clients_data = minmax_normalization(norm_clients_data)

for column in norm_clients_data:
    if sum(norm_clients_data[column]) < univers_tresh:
        norm_clients_data = norm_clients_data.drop(columns = column)

print(norm_clients_data)

             344       144       148       151       461       328       183  \
11377   0.010753  0.026882  0.016129  0.048387  0.284946  0.134409  0.048387   
30766   0.048193  0.072289  0.048193  0.000000  0.144578  0.012048  0.084337   
62982   0.076923  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
97964   0.139130  0.026087  0.017391  0.017391  0.104348  0.034783  0.095652   
61405   0.020000  0.240000  0.020000  0.060000  0.000000  0.180000  0.020000   
...          ...       ...       ...       ...       ...       ...       ...   
63427   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  1.000000   
147844  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
147454  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
144632  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
145875  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.250000   

             403       418       331  .

In [250]:
if do_Elbow:
    start = time()
    
    distorsions = []
    for k in range(Kmin, Kmax):
        print(k, end = ' ')
        model = KMeans(n_clusters=k)
        model.fit(norm_clients_data)
        distorsions.append(model.inertia_)

    fig = plt.figure(figsize=(15, 5))
    plt.plot(range(Kmin, Kmax), distorsions)
    plt.grid(True)
    plt.title('Elbow curve')
    plt.show()
    if save_fig:
        plt.savefig("UC2_fig/Elbow_Curve.jpg")
    # environ 10s*Kmax
    print("Time Taken to generate Elbow Curve : ~" + str(int(time() - start)) + "secondes")

In [251]:
if do_Elbow:
    start = time()

    distorsions = []
    for k in range(Kmin, Kmax):
        print(k, end = ' ')
        model = KMeans(n_clusters=k)
        model.fit(norm_clients_data)
        cluster = model.predict(norm_clients_data)
        cluster_centers = model.cluster_centers_

        treshold = Standard_Tresh if Standard_Norm else MinMax_Tesh
        NACK = 0
        for i in range(len(cluster_centers)):
            if sum([(1 if c > treshold else 0) for c in cluster_centers[i]]) == 0:
                NACK += cluster.tolist().count(i)

        distorsions.append(NACK/len(cluster))

    fig = plt.figure(figsize=(15, 5))
    plt.plot(range(Kmin, Kmax), distorsions)
    plt.grid(True)
    plt.title('Exclude Elbow curve')
    plt.show()
    
    if save_fig:
        plt.savefig("UC2_fig/Exclude_Elbow_Curve.jpg")

    # environ 10s*Kmax
    print("Time Taken to generate Exclude Elbow Curve : ~" + str(int(time() - start)) + "secondes")

In [252]:
start = time()
model = KMeans(K, n_init = n_init)
model.fit(norm_clients_data)
cluster = model.labels_
# environ 1.5s * n_init
print("Time Taken to fit model : ~" + str(int(time() - start)) + "secondes")

Time Taken to fit model : ~29secondes


In [246]:
cluster_centers = model.cluster_centers_

interest = {i:[] for i in range(len(cluster_centers))}

treshold = Standard_Tresh if Standard_Norm else MinMax_Tesh

for i in range(len(cluster_centers)):
    for j in range(len(cluster_centers[i])):
        if cluster_centers[i][j] > treshold:
            interest[i].append(clients_data.columns[j])

In [247]:
NACK = 0

conn = pyodbc.connect(login)
for key,values in interest.items():
    print("\n\nCluster", key, end=': ')
    proportion = cluster.tolist().count(key)*100/len(cluster)
    print(round(proportion,2),"%",)
    
    if len(values) == 0:
        NACK += proportion
        continue
    
    sql_command = '''SELECT [''' + TABLE_UNIVERS  + '''.KEY] AS Univ_KEY,
    [''' + TABLE_UNIVERS  + '''.Libelle_univers1_FR] AS Univ_1, 
    [''' + TABLE_UNIVERS  + '''.Libelle_univers2_FR] AS Univ_2, 
    [''' + TABLE_UNIVERS  + '''.Libelle_univers3_FR] AS Univ_3, 
    [''' + TABLE_UNIVERS  + '''.Libelle_univers4_FR] AS Univ_4
    FROM ['''+DB+'''].[DWH].[Dim_'''+TABLE_UNIVERS+''']
    WHERE '''
    for v in values:
        sql_command += "[" + TABLE_UNIVERS  + ".KEY] = " + str(v) + "\nOR "
    sql_command = sql_command[:-3]
    #print(sql_command)

    df_libele = pd.read_sql(sql_command, conn)
    print(df_libele.to_string(index=False))
conn.close()

print("\nNACK =", round(NACK,2),"%")



Cluster 0: 4.22 %
 Univ_KEY                     Univ_1                    Univ_2 Univ_3   Univ_4
      454 APERITIF   ENTREES   TAPAS BISCUITERIE SALEE  SNACKS  CHIPS > 1 PERS


Cluster 1: 1.49 %
 Univ_KEY   Univ_1 Univ_2 Univ_3    Univ_4
      250 BOISSONS   VINS FRANCE LANGUEDOC


Cluster 2: 2.13 %
 Univ_KEY         Univ_1       Univ_2        Univ_3        Univ_4
      299 PETIT DEJEUNER VIENNOISERIE VIENNOISERIES VIENNOISERIES
      437 PETIT DEJEUNER CAFE SERVICE  CAFE SERVICE  CAFE SERVICE


Cluster 3: 4.48 %
 Univ_KEY        Univ_1         Univ_2         Univ_3          Univ_4
      403 PRET A MANGER PLATS PREPARES TRADITION FOOD PLATS TRADITION
      406 PRET A MANGER PLATS PREPARES   ITALIAN FOOD  PLATS ITALIENS


Cluster 4: 5.35 %
 Univ_KEY         Univ_1                       Univ_2 Univ_3 Univ_4
      297 PETIT DEJEUNER BOULANGERIE   PROD CROQUANTS  PAINS  PAINS


Cluster 5: 12.73 %
 Univ_KEY         Univ_1       Univ_2        Univ_3        Univ_4
      299 PETIT DEJEUNER 

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqli

In [240]:
print("Total Time Taken: ~" + str(int(time() - Very_Start)) + "secondes") 

Total Time Taken: ~439secondes
